In [13]:
import torch.nn as nn
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric
from gnn_dataset import BipartiteNodeData, GraphDataset
from gnn_policy import GNNPolicy, BipartiteGraphConvolution, Critic
from tqdm import tqdm
from pathlib import Path
from acr_bb import ACRBBenv, instance_generator
from torch.distributions import Categorical
from tqdm import trange
from torch.autograd import Variable
from tqdm.notebook import tqdm
from helpers import get_graph_from_obs


class Policy(nn.Module):
    def __init__(self, init_policy_net, temperature, gamma=0.99):
        super().__init__()
        self.policy_net = init_policy_net
        self.temperature = temperature
        self.sf = nn.Softmax()
        self.gamma = gamma
        # Episode policy and reward history 
        self.policy_history = torch.Tensor()
        self.reward_episode = []
        
        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
        
    def forward(self, state, action_set):
        logits = self.policy_net(state.antenna_features, state.edge_index, state.edge_attr, state.variable_features)
        prob = Categorical(self.sf(logits[state.candidates]/self.temperature))
        action_id = prob.sample()
        
        # Add log probability of our chosen action to our history    
        if self.policy_history.dim() != 0:
            self.policy_history = torch.cat([self.policy_history, prob.log_prob(action_id.unsqueeze(dim=0))])
        else:
            self.policy_history = (prob.log_prob(action_id))
        return action_id
    

# add policy network
MAX_ITER = 1000
ACTOR_LR = 0.001
CRITIC_LR = 0.001
DEVICE = torch.device('cpu')

punishment_for_incomplete_episode = -1000
temperature = 5
GAMMA = 0.99

d = torch.load('trained_models/trained_params.pkl')
policy_net = GNNPolicy()
policy_net.load_state_dict( d )
policy = Policy(policy_net, temperature, gamma=GAMMA)

critic = Critic()
critic_loss_fn = nn.MSELoss()
env = ACRBBenv()
M = 4
N = 8
instances = instance_generator(M,N)
num_episodes = 2000

# def main(num_episodes):
running_reward = -1
avg_length = []

num_target_updates = 10 #@param {type: "integer"}
num_grad_steps_per_target_update = 10 #@param {type: "integer"}

optimizer_actor = torch.optim.Adam(policy.parameters(), lr=ACTOR_LR)
optimizer_critic = torch.optim.Adam(critic.parameters(), lr=CRITIC_LR)
# critic_update_frequency



def update_actor_critic(optimizer_actor, optimizer_critic, values):
    R = 0
    rewards = []
    baseline =[-300]*len(policy.reward_episode)
    
    for r in policy.reward_episode[::-1]:
        R = r + R
        rewards.insert(0, R)
    rewards = [rewards[0]]*len(policy.reward_episode)
    # Scale rewards
    print(rewards)
    rewards = torch.FloatTensor(rewards) - torch.FloatTensor(baseline)
    rewards = (rewards) / 300
    print(rewards[0])
    
#     # update critic
#     optimizer_critic.zero_grad()
#     critic_loss = critic_loss_fn(values, rewards)
#     critic_loss.backward()
#     optimizer_critic.step()
        
    
    # Update network weights
    optimizer_actor.zero_grad()
    actor_loss = (torch.sum(torch.mul(policy.policy_history, Variable(rewards)).mul(-1), -1))
    actor_loss.backward()
    optimizer_actor.step()
    
    #Save and intialize episode history counters
    policy.loss_history.append(actor_loss)
    policy.reward_history.append(np.sum(policy.reward_episode))
    policy.policy_history = Variable(torch.Tensor())
    policy.reward_episode= []
    



for i in tqdm(range(num_episodes)):
    instance = next(instances)
    values_history = torch.Tensor()
    obs, action_set, reward, done, _ = env.reset(instance)
    if done:
        policy.reward_episode.append(reward)
        continue

    for it in tqdm(range(MAX_ITER)):
        state = get_graph_from_obs(obs, action_set)
        value = critic(state)
        
        values_history = torch.cat([values_history, critic(state).unsqueeze(dim=0)])
    
        action_id = policy(state, action_set)
        # Step through environment using chosen action
        obs, action_set, next_reward, done, _ = env.step(action_set[action_id])
        
        # update the critic net
        # next_state = get_graph_from_obs(obs, action_set)
        # target = next_reward + critic(next_state)
        # optimizer_critic.zero_grad()
        # critic_loss = critic_loss_fn(critic(state), next_reward + critic(next_state))
        # critic_loss.backward()
        # optimizer_critic.step()
        
        # Save reward
        policy.reward_episode.append(next_reward)
        if done:
            break
    if not done:
        policy.reward_episode[-1] = punishment_for_incomplete_episode
    update_actor_critic(optimizer_actor, optimizer_critic, values_history )
    avg_length.append(it)
    if i % 10 == 0:
        print('Episode {}\tAverage length: {:.2f}'.format(i, np.mean(avg_length)))
        avg_length = []
torch.save( policy.policy_net.state_dict(), 'trained_models/unsupervised1.pkl')

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[-43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43, -43]
tensor(0.8567)
Episode 0	Average length: 42.00


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

[-830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -830, -83

  0%|          | 0/1000 [00:00<?, ?it/s]

[0.         4.0721078  5.59596191 1.67351686] [6.28318531 4.07210781 5.69413668 1.67351761]
[0.         4.0721078  5.59596191 1.67351686] [6.28318531 4.07210781 5.69413668 1.67351761]
[0.         4.0721078  5.59596191 1.67351686] [6.28318531 4.07210781 5.69413668 1.67351724]
[0.         4.0721078  5.6450493  1.67351724] [6.28318531 4.07210781 5.69413668 1.67351761]
[0.         4.0721078  5.6450493  1.67351724] [6.28318531 4.07210781 5.69413668 1.67351761]
[0.         4.0721078  5.6450493  1.67351724] [6.28318531 4.07210781 5.69413668 1.67351761]
[0.         4.0721078  5.6450493  1.67351724] [6.28318531 4.07210781 5.69413668 1.67351761]
[0.         4.0721078  5.60823376 1.67351705] [6.28318531 4.07210781 5.62050561 1.67351724]
[0.         4.0721078  5.60823376 1.67351705] [6.28318531 4.07210781 5.62050561 1.67351724]
[0.         4.0721078  5.60823376 1.67351714] [6.28318531 4.07210781 5.61436968 1.67351724]
[0.         4.0721078  5.60823376 1.67351714] [6.28318531 4.07210781 5.61436968 

IndexError: list index out of range

In [ ]:
## Actor Critic Implementation

In [1]:
import torch.nn as nn
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric
from gnn_dataset import BipartiteNodeData, GraphDataset, Experience
from gnn_policy import GNNPolicy, BipartiteGraphConvolution, Critic
from tqdm import tqdm
from pathlib import Path
from acr_bb import ACRBBenv, instance_generator
from torch.distributions import Categorical
from tqdm import trange
from torch.autograd import Variable
from tqdm.notebook import tqdm
from helpers import get_graph_from_obs

class Policy(nn.Module):
    def __init__(self, init_policy_net, temperature, gamma=0.99):
        super().__init__()
        self.policy_net = init_policy_net
        self.temperature = temperature
        self.sf = nn.Softmax()
        self.gamma = gamma
        # Episode policy and reward history 
        self.policy_history = torch.Tensor()
        self.reward_episode = []
        
        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
        
    def reset(self):
        self.policy_history = torch.Tensor()
        
    def forward(self, state, action_set):
        logits = self.policy_net(state.antenna_features, state.edge_index, state.edge_attr, state.variable_features)
        prob = Categorical(self.sf(logits[state.candidates]/self.temperature))
        action_id = prob.sample()
#         print(logits, action_id)
        # Add log probability of our chosen action to our history    
        if self.policy_history.dim() != 0:
            self.policy_history = torch.cat([self.policy_history, prob.log_prob(action_id.unsqueeze(dim=0))])
        else:
            self.policy_history = (prob.log_prob(action_id))
        return action_id
    

# add policy network
MAX_ITER = 1000
ACTOR_LR = 0.005
CRITIC_LR = 0.005
DEVICE = torch.device('cpu')

punishment_for_incomplete_episode = -1000
temperature = 5
GAMMA = 0.99

d = torch.load('trained_models/trained_params.pkl')
policy_net = GNNPolicy()
policy_net.load_state_dict( d )
policy = Policy(policy_net, temperature, gamma=GAMMA)

critic = Critic()
critic_loss_fn = nn.SmoothL1Loss()
env = ACRBBenv()
M = 4
N = 8
instances = instance_generator(M,N)
num_episodes = 2000

# def main(num_episodes):
running_reward = -1
avg_length = []

num_target_updates = 20
num_grad_steps_per_target_update = 20
num_actor_updates = 1

optimizer_actor = torch.optim.Adam(policy.parameters(), lr=ACTOR_LR)
optimizer_critic = torch.optim.Adam(critic.parameters(), lr=CRITIC_LR)


def update_actor_critic(optimizer_actor, optimizer_critic, experience):

    current_state, action, reward, next_state, terminal = experience.get_batch()
    for i in range(num_target_updates):
        with torch.no_grad():
            vs1 = critic(next_state).reshape([len(action), -1]).sum(dim=1)
            vs1[-1] = -1000
            vs1[terminal] = 0
        critic_target = reward + GAMMA*vs1

        
        for j in range(num_grad_steps_per_target_update):
            optimizer_critic.zero_grad()
            vs = critic(current_state).reshape([len(action), -1]).sum(dim=1)
            critic_loss = critic_loss_fn(critic_target.detach(), vs)
            critic_loss.backward()
            optimizer_critic.step()
        
    
    advantage = critic_target - vs
    print(advantage)
    # Update network weights
    optimizer_actor.zero_grad()
    actor_loss = (torch.sum(torch.mul(policy.policy_history, Variable(advantage)).mul(-1), -1))
    actor_loss.backward()
    optimizer_actor.step()
    
    
    
for i in tqdm(range(num_episodes)):
    instance = next(instances)
    policy.reset()
    experience = Experience()
    
    obs, action_set, reward, done, _ = env.reset(instance)

    if done:
        policy.reward_episode.append(reward)
        continue

    for it in tqdm(range(MAX_ITER)):
        state = get_graph_from_obs(obs, action_set)
        action_id = policy(state, action_set)
        # Step through environment using chosen action
        next_obs, action_set, next_reward, done, _ = env.step(action_set[action_id])
        
        experience.push(obs, action_id, next_reward, next_obs, done)
        obs = next_obs
        
        if done:
            break
        if next_reward < -5:
            break
     
    update_actor_critic(optimizer_actor, optimizer_critic, experience )
    
    avg_length.append(it)
    if i % 10 == 0:
        print('Episode {}\tAverage length: {:.2f}'.format(i, np.mean(avg_length)))
        avg_length = []
torch.save( policy.policy_net.state_dict(), 'trained_models/unsupervised1.pkl')

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[0.         6.08683577 3.04341788 5.39961237] [6.28318531 6.28318531 3.05568973 5.49778714]


/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


tensor([-3.6363e-01, -3.7523e-01, -3.7010e-01, -3.6669e-01, -3.6742e-01,
        -3.6643e-01, -3.6819e-01, -3.6885e-01, -3.6626e-01, -3.6740e-01,
        -3.6637e-01, -3.6861e-01, -3.6472e-01, -3.6637e-01, -3.6829e-01,
        -3.7005e-01, -3.6338e-01, -3.6618e-01, -3.7012e-01, -3.6597e-01,
        -3.6643e-01, -3.6850e-01, -3.6708e-01, -3.7016e-01, -3.6613e-01,
        -3.6656e-01, -3.6831e-01, -3.6720e-01, -3.6954e-01, -3.6749e-01,
        -3.5474e-01, -3.7857e-01, -3.5961e-01, -3.6004e-01, -3.7776e-01,
        -3.6745e-01, -3.6996e-01, -3.6436e-01, -3.6628e-01, -3.7120e-01,
        -3.6662e-01, -3.6665e-01, -3.6805e-01, -3.6445e-01, -3.7150e-01,
        -3.6753e-01, -3.6498e-01, -3.6885e-01, -3.6133e-01, -3.6725e-01,
        -3.6983e-01, -3.6707e-01, -3.6447e-01, -3.7085e-01, -3.6233e-01,
        -3.7142e-01, -3.6084e-01, -3.7362e-01, -3.6236e-01, -3.7254e-01,
        -3.6632e-01, -3.6564e-01, -3.6642e-01, -3.6581e-01, -3.7184e-01,
        -3.6584e-01, -3.6613e-01, -3.6552e-01, -3.6

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

torch.Size([20])


TypeError: reshape(): argument 'shape' (position 1) must be tuple of ints, not float

In [26]:
data.num_graphs

5

In [17]:
len(train_data)

8000

In [13]:
out

tensor(0.5404, grad_fn=<SigmoidBackward>)

In [3]:
out.shape

torch.Size([4])

In [3]:
state

BipartiteNodeData(antenna_features=[8, 3], edge_index=[2, 32], edge_attr=[32, 3], variable_features=[4, 9], candidates=[3], nb_candidates=3, candidate_choices=0, num_nodes=12)

In [7]:
torch.cat((torch.BoolTensor([]), torch.BoolTensor([a])))

tensor([True])

In [ ]:


for i in tqdm(range(num_episodes)):
    instance = next(instances)
    values_history = torch.Tensor()
    obs, action_set, reward, done, _ = env.reset(instance)
    if done:
        policy.reward_episode.append(reward)
        continue

    for it in tqdm(range(MAX_ITER)):
        state = get_graph_from_obs(obs, action_set)
        value = critic(state)
        
        values_history = torch.cat([values_history, critic(state).unsqueeze(dim=0)])
    
        action_id = policy(state, action_set)
        # Step through environment using chosen action
        obs, action_set, next_reward, done, _ = env.step(action_set[action_id])
        
        # update the critic net
        # next_state = get_graph_from_obs(obs, action_set)
        # target = next_reward + critic(next_state)
        # optimizer_critic.zero_grad()
        # critic_loss = critic_loss_fn(critic(state), next_reward + critic(next_state))
        # critic_loss.backward()
        # optimizer_critic.step()
        
        # Save reward
        policy.reward_episode.append(-next_reward)
        if done:
            break
    if not done:
        policy.reward_episode[-1] = punishment_for_incomplete_episode
    update_actor_critic(optimizer_actor, optimizer_critic, values_history )
    avg_length.append(it)
    if i % 10 == 0:
        print('Episode {}\tAverage length: {:.2f}'.format(i, np.mean(avg_length)))
        avg_length = []
torch.save( policy.policy_net.state_dict(), 'trained_models/unsupervised1.pkl')

In [ ]:
for i in range
validation_instances = 
def validate_policy():
    

## Evaluate Learnt Policy

In [9]:

import torch.nn as nn
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric
from gnn_dataset import BipartiteNodeData, GraphDataset
from gnn_policy import GNNPolicy, BipartiteGraphConvolution, Critic
from tqdm import tqdm
from pathlib import Path
from acr_bb import ACRBBenv, instance_generator
from torch.distributions import Categorical
from tqdm import trange
from torch.autograd import Variable
from tqdm.notebook import tqdm
from helpers import get_graph_from_obs


class Policy(nn.Module):
    def __init__(self, init_policy_net, temperature=1, gamma=1):
        super().__init__()
        self.policy_net = init_policy_net
        self.temperature = temperature
        self.sf = nn.Softmax()
        self.gamma = gamma
        # Episode policy and reward history 
        self.policy_history = torch.Tensor()
        self.reward_episode = []
        
        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
        
    def forward(self, state, action_set):
        logits = self.policy_net(state.antenna_features, state.edge_index, state.edge_attr, state.variable_features)
#         prob = Categorical(self.sf(logits[state.candidates]/self.temperature))
        action_id = torch.argmax(logits[state.candidates])
        
        # Add log probability of our chosen action to our history    
#         if self.policy_history.dim() != 0:
#             self.policy_history = torch.cat([self.policy_history, prob.log_prob(action_id.unsqueeze(dim=0))])
#         else:
#             self.policy_history = (prob.log_prob(action_id))
        return action_id

d = torch.load('trained_models/trained_params.pkl')
policy_net = GNNPolicy()
policy_net.load_state_dict( d )
policy = Policy(policy_net)

MAX_ITER = 1000
M = 4
N = 8
instances = instance_generator(M,N)

tree_sizes = []
for i in range(100):
    env = ACRBBenv()
    instance = next(instances)
    values_history = torch.Tensor()
    obs, action_set, reward, done, _ = env.reset(instance)
    if done:
        continue

    for it in tqdm(range(MAX_ITER)):
        state = get_graph_from_obs(obs, action_set)
        action_id = policy(state, action_set)
        obs, action_set, next_reward, done, _ = env.step(action_set[action_id])
        
        if done:
            tree_sizes.append(it)
            break
print(np.mean(tree_sizes))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

280.47


In [7]:
torch.argmax()

TypeError: argmax() missing 1 required positional arguments: "input"

In [ ]:
np.argmax

In [24]:
state

BipartiteNodeData(antenna_features=[8, 3], edge_index=[2, 32], edge_attr=[32, 3], variable_features=[4, 9], candidates=[3], nb_candidates=3, candidate_choices=0, num_nodes=12)

In [13]:
a.mul(-1)

tensor([ 0.6173,  0.3583, -1.5550, -1.1026, -1.9465])

In [18]:
torch.sum(a)

tensor(-3.6285)

In [2]:
state

BipartiteNodeData(antenna_features=[8, 3], edge_index=[2, 32], edge_attr=[32, 3], variable_features=[4, 9], candidates=[3], nb_candidates=3, candidate_choices=0, num_nodes=12)

In [7]:
values = torch.Tensor()
if values.dim() !=0:
    values = torch.cat([values, critic(state).unsqueeze(dim=0)])
else:
    values = critic(state)


In [6]:
values.dim()

1

In [20]:
targets = torch.FloatTensor([4,5])
values = torch.FloatTensor(values)

In [21]:
loss = critic_loss_fn(values, targets)

In [23]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [23]:
from gnn_policy import GNNPolicy, Critic
from acr_bb import instance_generator, ACRBBenv
from helpers import get_graph_from_obs

instances = instance_generator(4,8)

critic = Critic()

env = ACRBBenv()

instance = next(instances)
obs, action_set, reward, done, _ = env.reset(instance)

state = get_graph_from_obs(obs, action_set)
action_id = critic(state.antenna_features, state.edge_index, state.edge_attr, state.variable_features)

torch.Size([4, 64])
torch.Size([64])
tensor(0.1064, grad_fn=<SqueezeBackward1>)
